In [ ]:
import pandas as pd
import os

from sqlalchemy import create_engine, MetaData, Table
from dotenv import load_dotenv

from datetime import datetime, timedelta
today = datetime.now()


# --- 2. CONSTRUIR LA URL DE LA BASE DE DATOS DE FORMA SEGURA ---
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
yesterday = today - timedelta(days=1)
db_url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
load_dotenv()
engine = create_engine(db_url)


df_partidos_ayer = pd.read_sql(f"SELECT * FROM partido WHERE fecha = '{yesterday}'", engine)                    

In [2]:
df_jugadores = pd.read_sql(f"SELECT * FROM jugador where id_jugador < 11", engine)

In [8]:
valor_real = df_jugadores['valor_mercado'][1] / 15000000
valor_real = round(valor_real, 2)
valor_real


np.float64(3.27)